 
# Alpha and Beta Diversity

Different higher-level measures are often used to describe the microbiome in a sample. These do not provide information on changes in the abundance of specific taxa but allow us to access a broader change or difference in the composition of microorganisms. Alpha and beta diversity are examples of such measures.

Different measures exist to estimate diversity within a single sample, jointly called alpha diversity. The different measures reflect the richness (number) or distribution (evenness) of a microbial sample or aim to reflect a combination of both properties.

Rarefaction curves are often used when calculating alpha diversity indices because increasing numbers of sequenced taxa allow increasingly accurate estimates of total population diversity. Rarefaction curves can therefore be used to estimate the full sample richness, as compared to the observed sample richness.

While alpha diversity is a measure of microbiome diversity applicable to a single sample, beta diversity is a measure of the similarity or dissimilarity of two communities. As for alpha diversity, many indices exist, each reflecting different aspects of community heterogeneity. Key differences relate to how the indices value variation in rare species if they consider presence/absence only or incorporate abundance, and how they interpret shared absence. Bray-Curtis dissimilarity is a popular measure that considers both size (overall abundance per sample) and shape (abundance of each taxon) of the communities (Bray, 1957). Beta diversity is an essential measure for many popular statistical methods in ecology, such as ordination-based methods, and is widely used for studying the association between environmental variables and microbial composition.

In summary, alpha diversity measures can be seen as a summary statistic of a single population (within-sample diversity), while beta diversity measures are estimates of similarity or dissimilarity between populations (between samples).

**Source**: (https://biomcare.com/info/key-terms-in-microbiome-projects/)

### STEP : Diversity Analysis

Using QIIME2 to create diversity analisys graphs and calculations.

- [QIIME2 Workflow Overview](https://docs.qiime2.org/2022.8/tutorials/overview/)


#### Methods
- [diversity](https://docs.qiime2.org/2022.8/plugins/available/diversity/)
- [diversity alpha](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha/)
- [diversity alpha_phylogenetic](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-phylogenetic/)
- [diversity beta](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta/)
- [diversity core_metrics](https://docs.qiime2.org/2022.8/plugins/available/diversity/core-metrics/)
- [diversity alpha_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-group-significance/)
- [diversity beta_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta-group-significance/)
- [feature_table core_features](https://docs.qiime2.org/2022.8/plugins/available/feature-table/core-features/)
- [feature_table summarize](https://docs.qiime2.org/2022.8/plugins/available/feature-table/summarize/)
- [taxa filter-table](https://docs.qiime2.org/2022.8/plugins/available/taxa/filter-table/)
- [taxa collapse](https://docs.qiime2.org/2022.8/plugins/available/taxa/collapse/)

## Setup and settings

In [1]:
# Importing packages
import os
import pandas as pd
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata

from qiime2.plugins.phylogeny.pipelines import align_to_tree_mafft_fasttree

from qiime2.plugins.diversity.pipelines import alpha
from qiime2.plugins.diversity.pipelines import beta
from qiime2.plugins.diversity.pipelines import core_metrics
from qiime2.plugins.diversity.pipelines import alpha_phylogenetic

from qiime2.plugins.diversity.visualizers import alpha_group_significance
from qiime2.plugins.diversity.visualizers import beta_group_significance
from qiime2.plugins.diversity.visualizers import alpha_correlation
from qiime2.plugins.diversity.visualizers import beta_rarefaction

from qiime2.plugins.taxa.methods import filter_table
from qiime2.plugins.taxa.methods import collapse

from qiime2.plugins.feature_table.visualizers import tabulate_seqs
from qiime2.plugins.feature_table.visualizers import summarize
from qiime2.plugins.feature_table.visualizers import core_features
from qiime2.plugins.diversity.pipelines import core_metrics_phylogenetic

from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.feature_table.methods import filter_seqs

from qiime2.plugins.alignment.methods import mafft


import matplotlib.pyplot as plt

%matplotlib inline

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
base_dir = os.path.join('/', 'home')
metadata_file = os.path.abspath(os.path.join(base_dir, 'data', 'metadatada.tsv'))
experiment_name = ''
class_col = ''
replace_files = False

In [3]:
# Parameters
experiment_name = "ana-flavia-STD-NCxSTD-NR-trim"
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri"
manifest_file = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/manifest/manifest-ana-flavia-STD-NCxSTD-NR.csv"
metadata_file = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/metadata/metadata-ana-flavia-STD-NCxSTD-NR.tsv"
class_col = "group-id"
classifier_file = "/home/lauro/nupeb/rede-micro/models/silva-138-99-nb-classifier.qza"
top_n = 20
replace_files = False
phred = 20
trunc_f = 0
trunc_r = 0
overlap = 12
threads = 6
trim = {
    "overlap": 8,
    "forward_primer": "CCTACGGGRSGCAGCAG",
    "reverse_primer": "GGACTACHVGGGTWTCTAAT",
}


In [4]:
experiment_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name))
img_folder = os.path.abspath(os.path.join(experiment_folder, 'imgs'))

### Defining names, paths and flags

In [5]:
# QIIME2 Artifacts folder
qiime_folder = os.path.join(experiment_folder, 'qiime-artifacts')

# Input - DADA2 Artifacts
dada2_tabs_path = os.path.join(qiime_folder, 'dada2-tabs.qza')
dada2_reps_path = os.path.join(qiime_folder, 'dada2-reps.qza')
dada2_stat_path = os.path.join(qiime_folder, 'dada2-stat.qza')

# Input - Taxonaomic Artifacts
taxonomy_path = os.path.join(qiime_folder, 'metatax.qza')

# Create folder to store Alpha files
alpha_path = os.path.join(qiime_folder, 'alpha-analysis')
if not os.path.exists(alpha_path):
    os.makedirs(alpha_path)
    print(f'The new directory is created in {alpha_path}')
    
# Create folder to store Beta files
beta_path = os.path.join(qiime_folder, 'beta-analysis')
if not os.path.exists(beta_path):
    os.makedirs(beta_path)
    print(f'The new directory is created in {beta_path}')

# Output -Diversity Artifacts
alpha_diversity_path = os.path.join(alpha_path, 'alpha-diversity.qza')
alpha_diversity_view_path = os.path.join(alpha_path, 'alpha-diversity.qzv')
beta_diversity_path = os.path.join(beta_path, 'beta-diversity.qza')
beta_diversity_view_path = os.path.join(beta_path, 'beta-diversity.qzv')

In [6]:
def filter_and_collapse(tab, seqs, tax, meta, lvl, exclude=True, exclude_list='uncultured,unidentified,metagenome'):
    from qiime2.plugins.taxa.methods import collapse
    from qiime2.plugins.taxa.methods import filter_table
    from qiime2.plugins.feature_table.methods import filter_seqs
    from qiime2.plugins.feature_table.visualizers import summarize
    
    to_include = ('d', 'p', 'c', 'o', 'f', 'g', 's')[lvl-1]
    to_include += '__'
    to_exclude = exclude_list if exclude else None
    
    filtered_tabs = filter_table(
        table=tab, 
        taxonomy=tax,
        include=to_include,
        exclude=to_exclude,
        mode='contains').filtered_table
    
    filtered_seqs = filter_seqs(
        data = seqs,
        table = filtered_tabs,
    ).filtered_data
    
    collapsed_table = collapse(table=filtered_tabs, taxonomy=tax, level=lvl).collapsed_table
    collapsed_table_view = summarize(table=collapsed_table, sample_metadata=meta).visualization
    
    return collapsed_table, collapsed_table_view, filtered_seqs

## Step execution

### Load input files

This Step import the QIIME2 `FeatureTable[Frequency]` Artifact and the `Metadata` file.

In [7]:
#Load Metadata
metadata_qa = Metadata.load(metadata_file)

#Load FeatureTable[Frequency]
tabs = Artifact.load(dada2_tabs_path)
tabs_df = tabs.view(Metadata).to_dataframe().T

# FeatureData[Sequence]
reps = Artifact.load(dada2_reps_path)

# FeatureData[Taxonomy]
tax = Artifact.load(taxonomy_path)

In [8]:
# Filter FeatureTable[Frequency | RelativeFrequency | PresenceAbsence | Composition] based on Metadata sample ID values
tabs = filter_samples(
    table=tabs,
    metadata=metadata_qa,
).filtered_table
# Filter SampleData[SequencesWithQuality | PairedEndSequencesWithQuality | JoinedSequencesWithQuality] based on Metadata sample ID values; returns FeatureData[Sequence | AlignedSequence]
reps = filter_seqs(
    data=reps,
    table=tabs,
).filtered_data

{'min_frequency': 0, 'max_frequency': None, 'min_features': 0, 'max_features': None, 'metadata': Metadata
--------
14 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(type='categorical')
group-desc:  ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'where': None, 'exclude_ids': False, 'filter_empty_features': True, 'table': 2732 x 74 <class 'biom.table.Table'> with 12512 nonzero entries (6% dense)}
{'min_frequency': 0, 'max_frequency': None, 'min_features': 0, 'max_features': None, 'metadata': Metadata
--------
14 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(type='categorical')
group-desc:  ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'where': None, 'exclude_ids': False, 'filter_empty_features': True, 'table': 2732 x 74 <class 'biom.table.Table'> with 12512 nonzero entries (6% dense)}


{'metadata': None, 'where': None, 'exclude_ids': False, 'data': c688e3b1ada46ed57f5b7e7e0d56664f    (((T)), ((G)), ((A)), ((G)), ((G)), ((A)), ((A...
bc798de4a9acd3ff7ba51c244523be14    (((T)), ((G)), ((A)), ((G)), ((G)), ((A)), ((A...
fca5b1ccd94b107a3ddf3e99feaafb6f    (((T)), ((G)), ((A)), ((G)), ((G)), ((A)), ((A...
5ba9e679e692fcbf933c554317e03c5f    (((T)), ((A)), ((G)), ((G)), ((G)), ((A)), ((A...
04581eba9e6fd12787fb5948fcef030f    (((T)), ((G)), ((A)), ((G)), ((G)), ((A)), ((A...
                                                          ...                        
dbc83a2f59a61f2cc57971a85718406b    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
4393f0be1354f7198d1969c44d34d44a    (((T)), ((G)), ((A)), ((G)), ((G)), ((A)), ((A...
a86e1aaa4715133f8fa2c0f52e03e6d7    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
aa37170c6d59f9077a2644f8c25df05d    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
9c3f20cb7568feda60cc7776400d8d64    (((G)), ((A)), ((A)), ((A)), ((T)), ((G)

{'metadata': None, 'where': None, 'exclude_ids': False, 'data': c688e3b1ada46ed57f5b7e7e0d56664f    (((T)), ((G)), ((A)), ((G)), ((G)), ((A)), ((A...
bc798de4a9acd3ff7ba51c244523be14    (((T)), ((G)), ((A)), ((G)), ((G)), ((A)), ((A...
fca5b1ccd94b107a3ddf3e99feaafb6f    (((T)), ((G)), ((A)), ((G)), ((G)), ((A)), ((A...
5ba9e679e692fcbf933c554317e03c5f    (((T)), ((A)), ((G)), ((G)), ((G)), ((A)), ((A...
04581eba9e6fd12787fb5948fcef030f    (((T)), ((G)), ((A)), ((G)), ((G)), ((A)), ((A...
                                                          ...                        
dbc83a2f59a61f2cc57971a85718406b    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
4393f0be1354f7198d1969c44d34d44a    (((T)), ((G)), ((A)), ((G)), ((G)), ((A)), ((A...
a86e1aaa4715133f8fa2c0f52e03e6d7    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
aa37170c6d59f9077a2644f8c25df05d    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
9c3f20cb7568feda60cc7776400d8d64    (((G)), ((A)), ((A)), ((A)), ((T)), ((G)

## Alpha diversity analysis

#### Reference
- [The Use and Types of Alpha-Diversity Metrics in Microbial NGS](https://www.cd-genomics.com/microbioseq/the-use-and-types-of-alpha-diversity-metrics-in-microbial-ngs.html)
- [Alpha diversity metrics](http://scikit-bio.org/docs/0.2.0/generated/skbio.diversity.alpha.html)

#### Methods
- [diversity alpha](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha/): Computes a user-specified alpha diversity metric for all samples in a
feature table.
- [diversity alpha_phylogenetic](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-phylogenetic/): Computes a user-specified phylogenetic alpha diversity metric for all
samples in a feature table.
- [diversity alpha_correlation](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-correlation/): Determine whether numeric sample metadata columns are correlated with alpha diversity.
- [diversity alpha_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-group-significance/): Visually and statistically compare groups of alpha diversity values.

### Compute Alpha Diversity vectors
- [diversity alpha](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha/): Computes a user-specified alpha diversity metric for all samples in a feature table.
- [Alpha diversity metrics](http://scikit-bio.org/docs/0.2.0/generated/skbio.diversity.alpha.html)
 - Choices: ('ace', 'berger_parker_d', 'brillouin_d', 'chao1', 'chao1_ci', 'dominance', 'doubles', 'enspie', 'esty_ci', 'fisher_alpha', 'gini_index', 'goods_coverage', 'heip_e', 'kempton_taylor_q', 'lladser_pe', 'margalef', 'mcintosh_d', 'mcintosh_e', 'menhinick', 'michaelis_menten_fit', 'observed_features', 'osd', 'pielou_e', 'robbins', 'shannon', 'simpson', 'simpson_e', 'singles', 'strong')

In [9]:
metrics = ('ace', 'berger_parker_d', 'brillouin_d', 'chao1', 'chao1_ci', 'dominance', 'doubles', 'enspie', 'esty_ci', 'fisher_alpha', 'gini_index', 'goods_coverage', 'heip_e', 'kempton_taylor_q', 'lladser_pe', 'margalef', 'mcintosh_d', 'mcintosh_e', 'menhinick', 'michaelis_menten_fit', 'observed_features', 'osd', 'pielou_e', 'robbins', 'shannon', 'simpson', 'simpson_e', 'singles', 'strong')

# Sugestão de valores para Alpha diversity
# chao1 e observed_features (riqueza); shannon e simpson (diversidade - que levam em consideração riqueza e equitabilidade).
metrics = ('chao1', 'chao1_ci', 'observed_features', 'shannon', 'simpson', 'simpson_e')
alpha_diversities = dict()
for metric in metrics:
    print(f"Calculating alpha diversity: {metric}")
    try:
        alpha_diversity = alpha(table=tabs, metric=metric).alpha_diversity
        alpha_diversities[metric] = alpha_diversity
        # Save SampleData[AlphaDiversity] Artifact
        file_path = os.path.join(alpha_path, f'alpha-values-{metric}.qza')
        alpha_diversity.save(file_path)
        print(f"DONE: Calculating alpha diversity: {metric}")
    except Exception as e:
        print(f"ERROR: Calculating alpha diversity: {metric}")
        print(e)

Calculating alpha diversity: chao1
{'metric': 'chao1', 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>}
{'metric': 'chao1', 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
{'metric': 'chao1', 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
DONE: Calculating alpha diversity: chao1
Calculating alpha diversity: chao1_ci
{'metric': 'chao1_ci', 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>}
{'metric': 'chao1_ci', 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
{'metric': 'chao1_ci', 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
DONE: Calculating alpha diversity: chao1_ci
Calculating alpha diversity: observed_features
{'metric': 'observed_features', 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>}
{'table': 1263 x 14 <class 

DONE: Calculating alpha diversity: observed_features
Calculating alpha diversity: shannon
{'metric': 'shannon', 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>}
{'drop_undefined_samples': False, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
{'drop_undefined_samples': False, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
DONE: Calculating alpha diversity: shannon
Calculating alpha diversity: simpson
{'metric': 'simpson', 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>}
{'metric': 'simpson', 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
{'metric': 'simpson', 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
DONE: Calculating alpha diversity: simpson
Calculating alpha diversity: simpson_e
{'metric': 'simpson_e', 'table': <artifact: FeatureTable[Frequency] uuid: 

DONE: Calculating alpha diversity: simpson_e


### Create Phylogenetic inference

- [alignment align_to_tree_mafft_fasttree](https://docs.qiime2.org/2022.8/plugins/available/phylogeny/align-to-tree-mafft-fasttree/): Build a phylogenetic tree using fasttree and mafft alignment

This pipeline will start by creating a sequence alignment using MAFFT,
after which any alignment columns that are phylogenetically uninformative
or ambiguously aligned will be removed (masked). The resulting masked
alignment will be used to infer a phylogenetic tree and then subsequently
rooted at its midpoint. Output files from each step of the pipeline will be
saved. This includes both the unmasked and masked MAFFT alignment from
q2-alignment methods, and both the rooted and unrooted phylogenies from
q2-phylogeny methods.


Returns
- alignment : FeatureData[AlignedSequence] : The aligned sequences.
- masked_alignment : FeatureData[AlignedSequence] : The masked alignment.
- tree : Phylogeny[Unrooted] : The unrooted phylogenetic tree.
- rooted_tree : Phylogeny[Rooted] : The rooted phylogenetic tree.

In [10]:
mafft_alignment, mafft_masked_alignment, mafft_tree, mafft_rooted_tree = align_to_tree_mafft_fasttree(
    sequences=reps, n_threads=6, )

{'n_threads': 6, 'mask_max_gap_frequency': 1.0, 'mask_min_conservation': 0.4, 'parttree': False, 'sequences': <artifact: FeatureData[Sequence] uuid: cbc770df-79b0-4c79-b57a-0ad53a695640>}
{'n_threads': 6, 'parttree': False, 'sequences': <q2_types.feature_data._format.DNAFASTAFormat object at 0x7f461941f970>}
{'n_threads': 6, 'parttree': False, 'sequences': <q2_types.feature_data._format.DNAFASTAFormat object at 0x7f461941f970>}
Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 6 /tmp/qiime2-archive-k2prcrkp/cbc770df-79b0-4c79-b57a-0ad53a695640/data/dna-sequences.fasta



inputfile = orig
1263 x 430 - 253 d
nthread = 6
nthreadpair = 6
nthreadtb = 6
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
  401 / 1263 (thread    1)

 1201 / 1263 (thread    3)
done.

Constructing a UPGMA tree (efffree=0) ... 
 1260 / 1263
done.

Progressive alignment 1/2... 
STEP    58 / 1262 (thread    5) f

STEP   801 / 1262 (thread    0) f

STEP   901 / 1262 (thread    0) f
Reallocating..done. *alloclen = 1861
STEP  1201 / 1262 (thread    0) f

...

     10.41 seconds: ML split tests for    500 of   1229 internal splits
     10.55 seconds: ML split tests for    600 of   1229 internal splits


     10.70 seconds: ML split tests for    700 of   1229 internal splits
     10.84 seconds: ML split tests for    800 of   1229 internal splits


     10.98 seconds: ML split tests for    900 of   1229 internal splits
     11.12 seconds: ML split tests for   1000 of   1229 internal splits


     11.26 seconds: ML split tests for   1100 of   1229 internal splits
     11.40 seconds: ML split tests for   1200 of   1229 internal splits
Total time: 11.45 seconds Unique: 1232/1263 Bad splits: 0/1229


{'tree': <TreeNode, name: unnamed, internal node count: 1258, tips count: 1263>}
{'tree': <TreeNode, name: unnamed, internal node count: 1258, tips count: 1263>}


### Compute Alpha Diversity (Phylogeny)
- [diversity alpha_phylogenetic](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-phylogenetic/): Computes a user-specified phylogenetic alpha diversity metric for all samples in a feature table.
- Metrics: Choices ('faith_pd')

In [11]:
metrics = ('faith_pd', )
alpha_diversities_phylogenetic = dict()
for metric in metrics:
    print(f"Calculating alpha diversity: {metric}")
    try:
        alpha_diversity = alpha_phylogenetic(table=tabs, phylogeny=mafft_rooted_tree, metric=metric).alpha_diversity
        alpha_diversities_phylogenetic[metric] = alpha_diversity
        # Save Artifact
        file_path = os.path.join(alpha_path, f'alpha-phylogeny-{metric}.qza')
        alpha_diversity.save(file_path)
        print(f"DONE: Calculating alpha phylogeny: {metric}")
    except Exception as e:
        print(f"ERROR: Calculating alpha phylogeny: {metric}")

Calculating alpha diversity: faith_pd
{'metric': 'faith_pd', 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>, 'phylogeny': <artifact: Phylogeny[Rooted] uuid: eacd51b4-ceee-483b-8c5b-cac20c83ee17>}
{'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f46189c3ee0>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f461941fd30>}
{'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f46189c3ee0>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f461941fd30>}
DONE: Calculating alpha phylogeny: faith_pd


- [core-metrics-phylogenetic](https://docs.qiime2.org/2023.7/plugins/available/diversity/core-metrics-phylogenetic/)

In [12]:
s_depth = int(tabs.view(pd.DataFrame).sum(axis=1).min())
results = core_metrics_phylogenetic(
    table = tabs,
    phylogeny = mafft_rooted_tree,
    sampling_depth = s_depth,
    metadata = metadata_qa,
    n_jobs_or_threads = 6,
)

{'sampling_depth': 3381, 'metadata': Metadata
--------
14 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(type='categorical')
group-desc:  ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'n_jobs_or_threads': 6, 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>, 'phylogeny': <artifact: Phylogeny[Rooted] uuid: eacd51b4-ceee-483b-8c5b-cac20c83ee17>}
{'sampling_depth': 3381, 'metadata': Metadata
--------
14 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(type='categorical')
group-desc:  ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'with_replacement': False, 'n_jobs': 6, 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>}
{'sampling_depth': 3381, 'with_replacement': False, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero ent

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1761: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


{'n_jobs': 6, 'table': 1025 x 14 <class 'biom.table.Table'> with 2283 nonzero entries (15% dense)}
{'n_jobs': 6, 'table': 1025 x 14 <class 'biom.table.Table'> with 2283 nonzero entries (15% dense)}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f4618a0ffa0>}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f4618a0ffa0>}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f46189c0d30>}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f46189c0d30>}
{'metadata': Metadata
--------
14 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(type='categorical')
group-desc:  ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'custom_axes': None, 'ignore_missing_samples': False, 'ignore_pcoa_fea

{'metadata': Metadata
--------
14 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(type='categorical')
group-desc:  ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'custom_axes': None, 'ignore_missing_samples': False, 'ignore_pcoa_features': False, 'pcoa': <skbio.stats.ordination._ordination_results.OrdinationResults object at 0x7f461715f880>}


{'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f46189f3940>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f46193cafd0>}
{'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f46189f3940>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f46193cafd0>}
{'threads': 6, 'bypass_tips': False, 'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f46189c0e80>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f46189f36d0>}
{'threads': 6, 'bypass_tips': False, 'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f46189c0e80>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f46189f36d0>}
{'threads': 6, 'bypass_tips': False, 'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f4618a0f520>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f4618a0f550>}
{'threads': 6, 'bypass_tips': False, 'table': <q2_types.feature_table._format.BIOMV210Format

{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f461715fc40>}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f461715fc40>}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f461715f430>}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f461715f430>}
{'metadata': Metadata
--------
14 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(type='categorical')
group-desc:  ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'custom_axes': None, 'ignore_missing_samples': False, 'ignore_pcoa_features': False, 'pcoa': <skbio.stats.ordination._ordination_results.OrdinationResults object at 0x7f46189fd280>}
{'metadata': Metadata
--------
14 IDs x 3 columns
sample-name: ColumnProperties(type='

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.003381981502745507 and the largest is 0.5071312807777023.
  warn(


In [13]:
results_info = [("rarefied_table", "FeatureTable[Frequency]", "The resulting rarefied feature table."),
("faith_pd_vector", "SampleData[AlphaDiversity]", "Vector of Faith PD values by sample."),
("observed_features_vector", "SampleData[AlphaDiversity]", "Vector of Observed Features values by sample."),
("shannon_vector", "SampleData[AlphaDiversity]", "Vector of Shannon diversity values by sample."),
("evenness_vector", "SampleData[AlphaDiversity]", "Vector of Pielou's evenness values by sample."),
("unweighted_unifrac_distance_matrix", "DistanceMatrix", "Matrix of unweighted UniFrac distances between pairs of samples."),
("weighted_unifrac_distance_matrix", "DistanceMatrix", "Matrix of weighted UniFrac distances between pairs of samples."),
("jaccard_distance_matrix", "DistanceMatrix", "Matrix of Jaccard distances between pairs of samples."),
("bray_curtis_distance_matrix", "DistanceMatrix", "Matrix of Bray-Curtis distances between pairs of samples."),
("unweighted_unifrac_pcoa_results", "PCoAResults", "PCoA matrix computed from unweighted UniFrac distances between samples."),
("weighted_unifrac_pcoa_results", "PCoAResults", "PCoA matrix computed from weighted UniFrac distances between samples."),
("jaccard_pcoa_results", "PCoAResults", "PCoA matrix computed from Jaccard distances between samples."),
("bray_curtis_pcoa_results", "PCoAResults", "PCoA matrix computed from Bray-Curtis distances between samples."),
("unweighted_unifrac_emperor", "Visualization", "Emperor plot of the PCoA matrix computed from unweighted UniFrac."),
("weighted_unifrac_emperor", "Visualization", "Emperor plot of the PCoA matrix computed from weighted UniFrac."),
("jaccard_emperor", "Visualization", "Emperor plot of the PCoA matrix computed from Jaccard."),
("bray_curtis_emperor", "Visualization", "Emperor plot of the PCoA matrix computed from Bray-Curtis.")]

In [14]:
for i, info in enumerate(results_info):
    r_id, r_type, r_desc = info
    #print(i, r_id, r_type)
    file_name = f"{r_id}.qzv"
    if r_type == "FeatureTable[Frequency]":
        pass
    elif r_id.endswith('emperor'):
        print(i, r_id, r_type)
        print(f"--- {r_desc} ---")
        file_name = os.path.join(beta_path, file_name)
        print(f'Saving emperor file at: {file_name}\n')
        results[i].save(filepath=file_name)

13 unweighted_unifrac_emperor Visualization
--- Emperor plot of the PCoA matrix computed from unweighted UniFrac. ---
Saving emperor file at: /home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-STD-NCxSTD-NR-trim/qiime-artifacts/beta-analysis/unweighted_unifrac_emperor.qzv

14 weighted_unifrac_emperor Visualization
--- Emperor plot of the PCoA matrix computed from weighted UniFrac. ---
Saving emperor file at: /home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-STD-NCxSTD-NR-trim/qiime-artifacts/beta-analysis/weighted_unifrac_emperor.qzv



15 jaccard_emperor Visualization
--- Emperor plot of the PCoA matrix computed from Jaccard. ---
Saving emperor file at: /home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-STD-NCxSTD-NR-trim/qiime-artifacts/beta-analysis/jaccard_emperor.qzv

16 bray_curtis_emperor Visualization
--- Emperor plot of the PCoA matrix computed from Bray-Curtis. ---
Saving emperor file at: /home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-STD-NCxSTD-NR-trim/qiime-artifacts/beta-analysis/bray_curtis_emperor.qzv



### Alpha diversity correlation

This method only process `numeric` columns.


In [15]:
methods = ('spearman', 'pearson')
numerics_cols = metadata_qa.filter_columns(column_type='numeric')
if numerics_cols.column_count > 0:
    for metric, alpha_values in alpha_diversities.items():
        for method in methods:
            try:
                corr_view = alpha_correlation(alpha_diversity=alpha_values, metadata=numerics_cols, 
                                          method=method, intersect_ids=True).visualization
                view_path = os.path.join(alpha_path, f'alpha-correlation-{metric}-{method}.qzv')
                corr_view.save(view_path)
                corr_view
                print(f"DONE: Calculating alpha correlation: {metric} {method}")
            except Exception as e:
                print(f"ERROR: Calculating alpha correlation: {metric} {method}")

## Alpha diversity comparisons

Visually and statistically compare groups of alpha diversity values.

[diversity alpha_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-group-significance/)

In [16]:
for metric, alpha_values in alpha_diversities.items():
    print(f"Processing alpha_group_significance: {metric}")
    try:
        significance_view = alpha_group_significance(alpha_diversity=alpha_values, metadata=metadata_qa).visualization
        view_path = os.path.join(alpha_path, f'alpha-group-significance-{metric}.qzv')
        significance_view.save(view_path)
        significance_view
        print(f"DONE: Calculating alpha group significance: {metric}")
    except Exception as e:
        print(f"ERROR: Calculating alpha group significance: {metric}")

Processing alpha_group_significance: chao1
{'metadata': Metadata
--------
14 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(type='categorical')
group-desc:  ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'alpha_diversity': S210421121682    291.0
S210421121683    242.0
S210421121684    207.0
S210421121685    314.0
S210421121686    121.0
S210421121687    137.0
S210421121694    149.0
S210421121695    320.0
S210421121696    187.0
S210421121697    133.0
S210421121703     97.0
S210421121704     34.0
S210421121705    211.0
S210421121706    374.0
Name: chao1, dtype: float64}
DONE: Calculating alpha group significance: chao1
Processing alpha_group_significance: chao1_ci
ERROR: Calculating alpha group significance: chao1_ci
Processing alpha_group_significance: observed_features
{'metadata': Metadata
--------
14 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(

DONE: Calculating alpha group significance: observed_features
Processing alpha_group_significance: shannon
{'metadata': Metadata
--------
14 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(type='categorical')
group-desc:  ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'alpha_diversity': S210421121682    6.126624
S210421121683    6.103749
S210421121684    4.956948
S210421121685    6.145411
S210421121686    5.516588
S210421121687    5.179013
S210421121694    5.689236
S210421121695    6.177949
S210421121696    5.394384
S210421121697    5.432786
S210421121703    5.200061
S210421121704    3.786450
S210421121705    5.919258
S210421121706    6.381630
Name: shannon_entropy, dtype: float64}
DONE: Calculating alpha group significance: shannon
Processing alpha_group_significance: simpson


{'metadata': Metadata
--------
14 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(type='categorical')
group-desc:  ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'alpha_diversity': S210421121682    0.970384
S210421121683    0.973045
S210421121684    0.923226
S210421121685    0.967199
S210421121686    0.960541
S210421121687    0.935590
S210421121694    0.955916
S210421121695    0.969588
S210421121696    0.949858
S210421121697    0.958294
S210421121703    0.945950
S210421121704    0.859716
S210421121705    0.967740
S210421121706    0.969687
Name: simpson, dtype: float64}
DONE: Calculating alpha group significance: simpson
Processing alpha_group_significance: simpson_e
{'metadata': Metadata
--------
14 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(type='categorical')
group-desc:  ColumnProperties(type='categorical')

Call to_dataframe() for a tabular 

DONE: Calculating alpha group significance: simpson_e


## Beta diversity analysis

#### Reference
- [diversity beta](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta/): Computes a user-specified beta diversity metric for all pairs of samples in a feature table.
- [Beta diversity metrics](http://scikit-bio.org/docs/0.2.0/generated/skbio.diversity.beta.html)

- Metric Choices('aitchison', 'braycurtis', 'canberra', 'canberra_adkins', 'chebyshev', 'cityblock', 'correlation', 'cosine', 'dice', 'euclidean', 'hamming', 'jaccard', 'jensenshannon', 'kulsinski', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule')

In [17]:
metrics = ('aitchison', 'braycurtis', 'canberra', 'canberra_adkins', 'chebyshev', 'cityblock', 'correlation', 'cosine', 'dice', 'euclidean', 'hamming', 'jaccard', 'jensenshannon', 'kulsinski', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule')
metrics = ('euclidean', 'dice', 'braycurtis', 'correlation', 'cosine', 'matching', 'jaccard')
beta_diversities = dict()
for metric in metrics:
    print(f"Calculating beta diversity: {metric}")
    try:
        beta_diversity = beta(table=tabs, metric=metric, n_jobs=6, pseudocount=1).distance_matrix
        beta_diversities[metric] = beta_diversity
        # Save SampleData[BetaDiversity] Artifact
        file_path = os.path.join(beta_path, f'beta-values-{metric}.qza')
        beta_diversity.save(file_path)
        print(f"DONE: Calculating beta diversity: {metric}")
    except Exception as e:
        print(f"ERROR: Calculating beta diversity: {metric}")

Calculating beta diversity: euclidean
{'metric': 'euclidean', 'pseudocount': 1, 'n_jobs': 6, 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>}


{'metric': 'euclidean', 'pseudocount': 1, 'n_jobs': 6, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
{'metric': 'euclidean', 'pseudocount': 1, 'n_jobs': 6, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
DONE: Calculating beta diversity: euclidean
Calculating beta diversity: dice
{'metric': 'dice', 'pseudocount': 1, 'n_jobs': 6, 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>}
{'metric': 'dice', 'pseudocount': 1, 'n_jobs': 6, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
{'metric': 'dice', 'pseudocount': 1, 'n_jobs': 6, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
DONE: Calculating beta diversity: dice
Calculating beta diversity: braycurtis
{'metric': 'braycurtis', 'pseudocount': 1, 'n_jobs': 6, 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>}


{'n_jobs': 6, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
{'n_jobs': 6, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}


/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1761: DataConversionWarning: Data was converted to boolean for metric dice
  warnings.warn(msg, DataConversionWarning)


DONE: Calculating beta diversity: braycurtis
Calculating beta diversity: correlation
{'metric': 'correlation', 'pseudocount': 1, 'n_jobs': 6, 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>}
{'metric': 'correlation', 'pseudocount': 1, 'n_jobs': 6, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
{'metric': 'correlation', 'pseudocount': 1, 'n_jobs': 6, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
ERROR: Calculating beta diversity: correlation
Calculating beta diversity: cosine
{'metric': 'cosine', 'pseudocount': 1, 'n_jobs': 6, 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>}
{'metric': 'cosine', 'pseudocount': 1, 'n_jobs': 6, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
{'metric': 'cosine', 'pseudocount': 1, 'n_jobs': 6, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (

DONE: Calculating beta diversity: matching
Calculating beta diversity: jaccard
{'metric': 'jaccard', 'pseudocount': 1, 'n_jobs': 6, 'table': <artifact: FeatureTable[Frequency] uuid: 8777d539-c09d-4bdc-9ec9-c6b174dcd24e>}
{'n_jobs': 6, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}
{'n_jobs': 6, 'table': 1263 x 14 <class 'biom.table.Table'> with 2817 nonzero entries (15% dense)}


DONE: Calculating beta diversity: jaccard


/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1761: DataConversionWarning: Data was converted to boolean for metric matching
  warnings.warn(msg, DataConversionWarning)
/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1761: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


### Beta group significance

- [diversity beta_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta-group-significance/): Determine whether groups of samples are significantly different from one another using a permutation-based statistical test.
- Marti J Anderson. A new method for non-parametric multivariate analysis of variance. Austral ecology, 26(1):32–46, 2001. doi:https://doi.org/10.1111/j.1442-9993.2001.01070.pp.x.

In [18]:
methods = ('permanova', 'anosim', 'permdisp')
for method in methods:
    for metric, beta_diversity in beta_diversities.items():
        print(f'Calculating beta group significance with method {method} and metric {metric}')
        try:
            beta_view = beta_group_significance(distance_matrix=beta_diversity, 
                                                metadata=metadata_qa.get_column(class_col), 
                                                pairwise=True, method=method).visualization
            view_name = os.path.join(beta_path, f'beta-group-significance-{metric}-{method}.qzv')
            beta_view.save(view_name)
            print(f"DONE: Calculating beta group significance: {method} {metric}")
        except Exception as e:
            print(f"ERROR: Calculating beta group significance: {method} {metric}")

Calculating beta group significance with method permanova and metric euclidean


{'metadata': <CategoricalMetadataColumn name='group-id' id_count=14>, 'method': 'permanova', 'pairwise': True, 'permutations': 999, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f45fff7b940>}


DONE: Calculating beta group significance: permanova euclidean
Calculating beta group significance with method permanova and metric dice
{'metadata': <CategoricalMetadataColumn name='group-id' id_count=14>, 'method': 'permanova', 'pairwise': True, 'permutations': 999, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f4619623d00>}


DONE: Calculating beta group significance: permanova dice
Calculating beta group significance with method permanova and metric braycurtis
{'metadata': <CategoricalMetadataColumn name='group-id' id_count=14>, 'method': 'permanova', 'pairwise': True, 'permutations': 999, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f4618daa0a0>}


DONE: Calculating beta group significance: permanova braycurtis
Calculating beta group significance with method permanova and metric matching
{'metadata': <CategoricalMetadataColumn name='group-id' id_count=14>, 'method': 'permanova', 'pairwise': True, 'permutations': 999, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f4618daad90>}


...

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.0012090318022418473 and the largest is 0.1280774680018034.
  warn(


/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.0012090318022418473 and the largest is 0.1280774680018034.
  warn(


DONE: Calculating beta group significance: permdisp matching
Calculating beta group significance with method permdisp and metric jaccard
{'metadata': <CategoricalMetadataColumn name='group-id' id_count=14>, 'method': 'permdisp', 'pairwise': True, 'permutations': 999, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f4618b42a90>}


DONE: Calculating beta group significance: permdisp jaccard


<Figure size 432x288 with 0 Axes>

### Beta group Rarefaction

- [diversity beta_rarefaction](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta-rarefaction/): Repeatedly rarefy a feature table to compare beta diversity results within a given rarefaction depth.  For a given beta diversity metric, this visualizer will provide: an Emperor jackknifed PCoA plot, samples clustered by UPGMA or neighbor joining with support calculation, and a heatmap showing the correlation between rarefaction trials of that beta diversity metric.